On cherche à réecrire le checkpoint pth de SAM pour être "conforme" à lightning, i.e avec les clés attendues dans le dictionnaire de state_dict.

L'erreur initiale est : `KeyError: 'pytorch-lightning_version'`

In [7]:
import hydra
import torch
from src.commons.constants import PROJECT_PATH
from omegaconf import DictConfig, OmegaConf

In [2]:
ls ../../configs

callbacks/  eval.yaml    hparams_search/  local/    model/     trainer/
data/       experiment/  hydra/           logger/   paths/     train.yaml
debug/      extras/      __init__.py      metrics/  sam_type/


In [3]:
def load_config():
    # Initialize the Hydra configuration
    hydra.initialize(config_path="../../configs", version_base=None)
    
    # Compose the configuration with the desired environment override
    cfg = hydra.compose(config_name="train", overrides=["experiment=mp_naive", "sam_type=small", "data=levir-cd"])
    
    return cfg

In [4]:
from hydra.core.global_hydra import GlobalHydra
GlobalHydra.instance().clear()
cfg = load_config()
print(OmegaConf.to_yaml(cfg))

data:
  name: levir-cd
  _target_: src.data.datamodule.CDDataModule
  params:
    prompt_type: sample
    n_prompt: 1
    loc: center
    batch_size: 2
    n_shape: null
    num_worker: 8
model:
  network:
    image_encoder:
      _target_: src.models.segment_anything.modeling.image_encoder_dev.ImageEncoderViT
      depth: 12
      embed_dim: 768
      img_size: 1024
      mlp_ratio: 4
      norm_layer: null
      num_heads: 12
      patch_size: 16
      qkv_bias: true
      use_rel_pos: true
      global_attn_indexes:
      - 2
      - 5
      - 8
      - 11
      window_size: 14
      out_chans: 256
    prompt_encoder:
      _target_: src.models.segment_anything.modeling.prompt_encoder_dev.PromptEncoder
      embed_dim: 256
      image_embedding_size:
      - 64
      - 64
      input_image_size:
      - 1024
      - 1024
      mask_in_chans: 16
    mask_decoder:
      transformer:
        _target_: src.models.segment_anything.modeling.transformer_dev.TwoWayTransformer
        depth:

In [5]:
module = hydra.utils.instantiate(cfg.model.instance)

In [8]:
path = "/var/data/usr/mdizier/stylo_magique/checkpoints/sam/sam_vit_b_01ec64.pth"
module.model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [ ]:
module